# CBurnFS

    a = CBurnFS(bootpath)
    
CBurnFS creates a wrapper around an instance of `MulticelFS` and manages its configuration.

### Management Role

CBurnFS adds and removes members from the `MulticelFS` and configures authentication. It reads and writes state in the `@` provided by the multicel.

### Delegation

The `MulticelFS` is responsible for combining multiple domains and for reporting item membership in the `getinfo()` output.

### Metadata

The 
    
## Init

For each entry in `bootpath`/`.cosm/etc/fstab`, create a Dcel.

    -> dcel_list

Then create a `Dcel(dcel_list, MulticelFS)`

    -> root

Set `root` as the CBurnFS instance's root resource.

    self.root = root

Rock and roll.

## File-change Notifications

File notifications are PLANNED to be implemented as follows:

Keeps a registry of *tags*.

For each tag in the registry, there can be one or more *listeners*.

A listener is notified over a channel.

A an example remote listener: `https://api.example.com/file-did-update`

would receive whatever message was assigned to the tag for a given flow.

Ie. as part of some subject's metadata:

    flow:
        some-tag: message .
        
Would result in:

    POST: { some-tag: [ message, url-of-subject ] }
    
An example local listener: `callback`

Would result in:

    callback(message,url-of-subject)
    
### Listener Objects

A listener object could wrap any sort of communication channel. The object would then be registered to the tag-registry, and it's `notify()` method would be called when appropriate.

    class MyListener():
        def notify(message, subject_url):
            # process

### Notify on Change

In order to implement notifications when files are changed, all FS methods which create change must be overridden. This will be a work in progress.

The most obvious method that needs to be implemented is writetext() since it could be used to write the 'fstab' file, and thereby trigger `_reinit()` within the filesystem.


## Concurrency

There may be certain expectations about file system commands and their results, about how they are anticipated by the user to behave concurrently.

### Blocking and Resource Busy

Given a single user at a single machine, using a terminal, a file system command is issued, the terminal blocks until it is complete.

Another user, logged in via ssh, may see that a resource is busy, but would not know what operation was being performed on the resource. This user, however, would still expect to have access to the file system. Other resources within it would be expected to be available.

### CBurnFS's expectations for blocking

User 1 loads a resource from the web. The page-load is the blocking operation.

User 2 loads another resource from the web. Does not expect that User 1's page-load would block User 2.

The most important blocking request types that can be expected are directories, and file contents.

    USERAGENT ---request---> WEBSERVER ---request---> FILESYSTEM ---
                                                                    |
    USERAGENT <--response--- WEBSERVER <--response------------------

#### Non-blocking

User 1 sets a property of a file or directory via webui. The user expects the update to happen in the background while the user continues to use the page. The user would also like to be able to navigate to another page without necessarily waiting for the property update to complete.


#### Pending Progress

When a non-blocking file system task is started, a `progess_id` is created and a `progress` item is added into the getinfo content of the file.

    {
      'progress': {
        'updateHosts': {
          {{ hostname }}: {{ progress_id }}
        },
        'removeHosts': {
          {{ hostname }}: {{ progress_id }}
        }
      }
    }

In this way, multiple calls to 'updateHosts' with the same hostname will share a single result -- if the updateHosts() handler checks for an existing `progress_id`. The CBurnFS needs to keep track of hung or vaporized tasks. 

    {
      'task-map': {
        {{ progress_id }}: {{ task_handle }}
      }
    }

When the task completes, the `progress` item is removed and the `progress_id` is added to a `completed` stack.

    { 'completed': { progress_id: final_status } }
    

In [2]:
from fs.osfs import OSFS
from fs.base import FS
from fs.info import Info
from fs.errors import ResourceNotFound
from fs.copy import copy_dir, copy_file

from MulticelFS import MulticelFS
from Dcel import Dcel # TODO: factor out
from APath import APath
from Fudge import Fudge
from metafs import MetaFS
from metafs_proxy import MetaFSProxy

from copy import deepcopy
import json
import six
from urllib.parse import urlparse
from os.path import split
if six.PY2:
    from urllib import unquote
else:
    from urllib.parse import unquote
from threading import Thread

#for merge lib (tbd): mergeinfo


_version='0.18'

# Adding MetaFSProxy to store custom metadata.

# --- version 0.16 goals INHERIT
# Refactoring to simplify dependencies.
# Goal is an independent, installable python module.

# usage:
# cbfs = CBurnFS(bootpath)

def init_dcel_from_url(boot,url):
    dc = None
    return dc

FSTAB_RELPATH = '@/etc/fstab'
FSTAB_ABSPATH = '/@/etc/fstab'

def metafs_set_progress(metafs, path, functionName, host, status):
    info = metafs.getinfo(path).raw
    info.update({"progress":{functionName:{host:status}}})
    metafs.setinfo(path,info)
    
def metafs_remove_progress(metafs, path, functionName, host):
    info = metafs.getinfo(path).raw
    del(info['progress'][functionName][host])
    if len(info['progress'][functionName]) == 0:
        del(info['progress'][functionName])
    if len(info['progress']) == 0:
        del(info['progress'])
    metafs.setinfo(path,info)

def copy_executor(metafs, path, host, copy_func, fs1, path1, fs2, path2):
    metafs_set_progress(metafs, path, "updateHosts", host, "pending")
    copy_func(fs1, path1, fs2, path2)
    metafs_remove_progress(metafs, path, "updateHosts", host)
    
    
def remover_executor(metafs, path, host, remover_func, func_self, target_path):
    metafs_set_progress(metafs, path, "removeHosts", host, "pending")
    remover_func(target_path)
    metafs_remove_progress(metafs, path, "removeHosts", host)


class CBurnFS(APath):        
    
    def __loadFstab(self, bootpath: str):
        boot = Fudge(bootpath)
        # FIXME: must force load of root - consider this a bug
        boot_root = boot/'/'
        fstab = boot/FSTAB_RELPATH
        cels = []
        metafs_conf = dict()
        for ea in fstab:
            """
            The fstab parser is located in the APath Cosm.
            It might not handle tabs. Try to use spaces only.
            """
            if ea/'vfstype' == 'cburnfs':
                urlstr = ea/'spec'
                _scheme = ea/'spec.url/scheme'
                if _scheme == '':
                    _scheme = 'file'
                service_class = boot._apath.cosm['services'][str(_scheme)].value
                try:
                    cels += [ Dcel(address=str(urlstr), 
                                service_class=service_class) ]
                except:
                    pass
            if ea/'vfstype' == 'cburnfs-meta':
                '''
                The last 'cburn-metafs' defined will succeed.
                '''
                host_and_port = str(ea/'spec.url/netloc').split(':')
                metafs_conf['REDIS_CONTAINER_NAME'] = host_and_port[0]
                if len(host_and_port) > 1:
                    metafs_conf['REDIS_CONTAINER_PORT'] = int(host_and_port[1])
                else:
                    metafs_conf['REDIS_CONTAINER_PORT'] = 6379
                try:
                    metafs_conf['USERPUBLICID'] = str(ea/'mntopts.cskvp/userid')
                except:
                    metafs_conf['USERPUBLICID'] = 'unset'
                try:
                    metafs_conf['USERHOMENAME'] = str(ea/'mntopts.cskvp/userhome')
                except:
                    metafs_conf['USERHOMENAME'] = 'unset'
                try:
                    metafs_conf['USERFSURL'] = str(ea/'mntopts.cskvp/userurl')
                except:
                    metafs_conf['UESRFSURL'] = 'unset'
                
        root = Dcel(
            service_class=MulticelFS,
            address=cels
        )
        meta = MetaFS(config=metafs_conf)
        return root, meta
                
    def __init__(self, bootpath: str):
        self._bootpath = bootpath
        root, meta = self.__loadFstab(bootpath)
        self.metafs = meta
        metaproxy = MetaFSProxy(root,meta)
        super().__init__(Dcel(service=metaproxy))
        self._init_listener_system()
        
    def _reinit(self):
        print(f"CBurnFS::_reinit() called.")
        root = self.__loadFstab(self._bootpath)
        super()._reinit(root)
        
        
    # updater magic
        
    def updateHosts(self,path,hosts):
        
        metaproxy = self.target.service
        metafs = metaproxy.metafs
        dcel = metaproxy.targetfs   # note: unsyncs metafs until next getinfo()
        svc = dcel.service
        
        _path = unquote(path)
        pathDir,pathBase = split(_path)
        
        if type(svc) == MulticelFS:   
            for host in hosts:
                
                # get host fs
                dest = svc.get_dcel_by_host(host)
            
                # make path on host fs
                destSvc = dest.service
                destDirFS = destSvc.makedirs(pathDir,None,True)
                
                # copy path from multifs
                #   to path in host fs
                if svc.getinfo(_path).is_dir:
                    copy_func = copy_dir
                else:
                    copy_func = copy_file
                    
                # WIP This assumes that svc and destDirFS
                # are thread-safe.
                copy_thread = Thread(target=copy_executor,
                    args=[metafs,path,host,
                          copy_func,svc,_path,destDirFS,pathBase]
                )
                copy_thread.start()

    def removeHosts(self,path,hosts):
        
        metaproxy = self.target.service
        metafs = metaproxy.metafs
        dcel = metaproxy.targetfs   # note: unsyncs metafs until next getinfo()
        svc = dcel.service
        
        _path = unquote(path)
        pathDir,pathBase = split(_path)
        
        if type(svc) == MulticelFS:   
            for host in hosts:
                # get host fs
                dest = svc.get_dcel_by_host(host)
        
                if dest.getinfo(_path).is_dir:
                    remover_func = dest.service.removetree
                else:
                    remover_func = dest.service.remove
                    
                # WIP This assumes that dest.service
                # is thread-safe.
                remover_thread = Thread(target=remover_executor,
                    args=[metafs,path,host,
                          remover_func,dest.service,_path]
                )
                remover_thread.start()
                    
                    
    def urlListFromDict(self, path, ob) -> list:
        dirlist = list()
        if type(ob) is dict:
            for each in ob:
                dirlist = dirlist + self.urlListFromDict(f"{path}/{each}", ob[each])
        else:
            return [(path, ob)]
        return dirlist

    def updateMultiValue(self, path, multiValue):
        """multiValue should already have been parsed."""
        _path = unquote(path)
        actionPairs = self.urlListFromDict(_path, multiValue)
        root_fu = Fudge(self)
        for url,val in actionPairs:
            (root_fu/url)['.'] = val
    
    # webdav style updaters
    
    def propertyupdate(self,path,uprq):
        
        print(f"CBurnFS::propertyupdate(): path={path}, uprq={uprq}")
        
        for verb in uprq:
            if verb == 'append':
                if "cburn" in uprq[verb]:
                    target = uprq[verb]["cburn"]
                    if "hosts" in target:
                        self.updateHosts(
                            path,
                            target["hosts"])
                    if "multivalue" in target:
                        self.updateMultiValue(
                            path,
                            target["multivalue"])
                        if(path.startswith(FSTAB_RELPATH)
                           or path.startswith(FSTAB_ABSPATH)):
                            self._reinit()
            
            if verb == 'remove':
                if "cburn" in uprq[verb]:
                    target = uprq[verb]["cburn"]
                    if "hosts" in target:
                        self.removeHosts(path,
                            target["hosts"])
                
        return 'CBurnFS.propertyupdate():return: improve this response'
    
    def processRequest(self,path,rq):
        # rq is a dict object from flask app.
        print(f"CBurnFS::processRequest() rq={rq}")
        response = dict()
        for mod in rq:
            if mod == 'propertyupdate':
                response[mod] = self.propertyupdate(path,rq[mod])
        return response
    
    # ---- listener notification system ----
    
    def _init_listener_system(self):
        self._flow = {'listen':dict()}
        
    def add_listener(self,tag,listener):
        tags = self._flow['listen']
        if not tag in tags:
            tags[tag] = []
        tags[tag].append = listener
        
    def remove_listener(self,tag,listener):
        tags = self._flow['listen']
        if not tag in tags:
            return
        if listener in tags[tag]:
            tags[tag].remove(listener)
            
    # ---- FS shadow methods ----
    
    def writetext(self, path=None, contents='', encoding='utf-8',
                  errors=None, newline=''):
        # dirty hook to avoid fleshing out listener system
        if path == FSTAB_ABSPATH:
            old_content = self.readtext(path)
        if not old_content == contents:
            super().writetext(path,contents,encoding,errors,newline)
            self._reinit()
        

## Todo

- cburnfs: config location
- cburnfs: check for presence of '@/etc/fstab' and error informatively
- demodata: location, generator
- blackstrap: yaml config
- hiena: yaml grammar definitions

### Make it Just Work

    cbfs = CBurnFS('.')
    
Should initialize a `CBurnFS` rooted at the input path.

Lookup of @/etc/fstab should fail silently
-- or at least provide an educational message that 
the file system is running on a single layer.

Adding/Modifying an @/etc/fstab should trigger a
re-build of the layers.

Adding/Modifying other aspects of @ should modify
the Apath Cosm --- provide for extending the services
for URL's -- such as s3, file, sftp etc.


### SETUP TEST ENVIRONMENT

In [4]:
# This needs to be tweaked for your environment.
# It references mount locations that may not exist.
import demo_blackstrap_config

In [5]:
# TODO: need to check for presence of '@/etc/fstab' and fail gracefully
cbfs = CBurnFS('file://fs.localhost')

ConnectionError: Error 111 connecting to localhost:6379. Connection refused.

In [ ]:
cbfs.writetext('/test.txt','Hola, Amigo!')

In [ ]:
cbfs.readtext('/test.txt')

### TEST removeHosts via propertyupdate()

In [5]:
import json

In [6]:
path = '/fruit/apples.txt'
cbfs.getinfo(path).raw

{'basic': {'name': 'apples.txt', 'is_dir': False},
 'hosts': ['file://fs.localhost']}

In [6]:
updateJSON = '{"propertyupdate":{"remove":{"cburn":{"hosts":["file://fs4.localhost"]}}}}'
updateRequest = json.loads(updateJSON)

In [8]:
print(cbfs.getinfo(path).raw)
cbfs.processRequest(path,updateRequest)
print(cbfs.getinfo(path).raw)

{'basic': {'name': 'apples.txt', 'is_dir': False}, 'hosts': ['file://fs.localhost']}


### TEST updateHosts via propertyupdate()

In [10]:
import json

In [11]:
path = 'fruit/apples.txt'
cbfs.getinfo(path).raw

{'basic': {'name': 'apples.txt', 'is_dir': False},
 'hosts': ['file://fs.localhost']}

In [12]:
updateJSON = '{"propertyupdate":{"append":{"cburn":{"hosts":["file://fs4.localhost"]}}}}'
updateRequest = json.loads(updateJSON)

In [13]:
print(cbfs.getinfo(path).raw)
cbfs.processRequest(path,updateRequest)
print(cbfs.getinfo(path).raw)

{'basic': {'name': 'apples.txt', 'is_dir': False}, 'hosts': ['file://fs.localhost']}
CBurnFS::processRequest() rq={'propertyupdate': {'append': {'cburn': {'hosts': ['file://fs4.localhost']}}}}
CBurnFS::propertyupdate(): path=fruit/apples.txt, uprq={'append': {'cburn': {'hosts': ['file://fs4.localhost']}}}
{'basic': {'name': 'apples.txt', 'is_dir': False}, 'hosts': ['file://fs.localhost', 'file://fs4.localhost']}


### TEST _reinit() HOOK ON propertyupdate()

In [26]:
import json

In [27]:
# propertyupdate JSON
updateJSON = '{"propertyupdate":{"append":{"cburn":{"multivalue":{"1":{"spec":"file://fs4.localhost"},"2":{"mntopts.cskvp":{"shortid":"FS1"}},"3":{"mntopts.cskvp":{"shortid":"FS3"}}}}}}}'
updateRequest = json.loads(updateJSON)
path = "@/etc/fstab.fstab"
cbf = Fudge(cbfs)
fstab = cbf/path

In [28]:
print(fstab/'1/spec')

file://fs4.localhost


In [29]:
cbfs.processRequest(path,updateRequest)

CBurnFS::processRequest() rq={'propertyupdate': {'append': {'cburn': {'multivalue': {'1': {'spec': 'file://fs4.localhost'}, '2': {'mntopts.cskvp': {'shortid': 'FS1'}}, '3': {'mntopts.cskvp': {'shortid': 'FS3'}}}}}}}
CBurnFS::propertyupdate(): path=@/etc/fstab.fstab, uprq={'append': {'cburn': {'multivalue': {'1': {'spec': 'file://fs4.localhost'}, '2': {'mntopts.cskvp': {'shortid': 'FS1'}}, '3': {'mntopts.cskvp': {'shortid': 'FS3'}}}}}}
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
CBurnFS::_reinit() called.


{'propertyupdate': 'CBurnFS.propertyupdate():return: improve this response'}

### TEST MulticelFS via getinfo()

In [12]:
i = cbfs.getinfo('/fruit/cherries.html.txt')

In [13]:
print(i.raw)

{'basic': {'name': 'cherries.html.txt', 'is_dir': False}, 'hosts': ['file://fs3.localhost']}


### TEST CBurnFS::propertyupdate() and updateMultiValue()

In [15]:
import json

In [16]:
# propertyupdate JSON
updateJSON = '{"propertyupdate":{"append":{"cburn":{"multivalue":{"1":{"mntopts.cskvp":{"shortid":"fs2"}},"2":{"mntopts.cskvp":{"shortid":"fs1"}},"3":{"mntopts.cskvp":{"shortid":"fs3"}}}}}}}'
updateRequest = json.loads(updateJSON)
print(updateRequest)

{'propertyupdate': {'append': {'cburn': {'multivalue': {'1': {'mntopts.cskvp': {'shortid': 'fs2'}}, '2': {'mntopts.cskvp': {'shortid': 'fs1'}}, '3': {'mntopts.cskvp': {'shortid': 'fs3'}}}}}}}


In [15]:
# locate path relative to the above request
path = "@/etc/fstab.fstab"
cbf = Fudge(cbfs)
fstab = cbf/path
print(fstab)

{'1': {'spec': <Dcel.Dcel object at 0x7fa9df966fa0>, 'file': <Dcel.Dcel object at 0x7fa9df966580>, 'vfstype': <Dcel.Dcel object at 0x7fa9df966a90>, 'mntopts': <Dcel.Dcel object at 0x7fa9df966ca0>, 'freq': <Dcel.Dcel object at 0x7fa9df966970>, 'passno': <Dcel.Dcel object at 0x7fa9df9660a0>}, '2': {'spec': <Dcel.Dcel object at 0x7fa9df9664f0>, 'file': <Dcel.Dcel object at 0x7fa9df966d30>, 'vfstype': <Dcel.Dcel object at 0x7fa9df966550>, 'mntopts': <Dcel.Dcel object at 0x7fa9df966280>, 'freq': <Dcel.Dcel object at 0x7fa9df966430>, 'passno': <Dcel.Dcel object at 0x7fa9df966520>}, '3': {'spec': <Dcel.Dcel object at 0x7fa9df9b5130>, 'file': <Dcel.Dcel object at 0x7fa9df9b5e80>, 'vfstype': <Dcel.Dcel object at 0x7fa9df9b5fd0>, 'mntopts': <Dcel.Dcel object at 0x7fa9df9b5610>, 'freq': <Dcel.Dcel object at 0x7fa9df9b5850>, 'passno': <Dcel.Dcel object at 0x7fa9df9b5100>}}


In [18]:
cbfs.processRequest(path,updateRequest)

CBurnFS::processRequest() rq={'propertyupdate': {'append': {'cburn': {'multivalue': {'1': {'mntopts.cskvp': {'shortid': 'fs2'}}, '2': {'mntopts.cskvp': {'shortid': 'fs1'}}, '3': {'mntopts.cskvp': {'shortid': 'fs3'}}}}}}}
CBurnFS::propertyupdate(): path=@/etc/fstabtest.fstab, uprq={'append': {'cburn': {'multivalue': {'1': {'mntopts.cskvp': {'shortid': 'fs2'}}, '2': {'mntopts.cskvp': {'shortid': 'fs1'}}, '3': {'mntopts.cskvp': {'shortid': 'fs3'}}}}}}
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
Fudge::__setitem__() target.value type: <class 'Dcel.Dcel'>
CBurnFS::_reinit() called.


{'propertyupdate': 'CBurnFS.propertyupdate():return: improve this response'}

### TEST CBurnFS File Functions

In [19]:
cbfs.getinfo('/boats/skiff.txt').raw

{'basic': {'name': 'skiff.txt', 'is_dir': False},
 'hosts': ['file://fs.localhost', 'file://fs4.localhost']}

In [20]:
cbfs.listdir('/')

['.cosm', 'test.txt', 'fruit', '.cloud', '@', 'boats', '..@', 'numbers']

In [21]:
cbf = Fudge(cbfs)

In [22]:
for each in cbf/'fruit':
    print(each)

red
<i>sakuranbo</i> means <b>cherries</b>



### DEVELOP INIT PROCEDURE

In [4]:
boot = Fudge("file://fs.localhost")
fstab = boot/'.cosm/etc/fstab.fstab/'
cels = []
for ea in fstab:
    if ea/'vfstype' == 'cburnfs':
        urlstr = ea/'spec'
        _scheme = ea/'spec.url/scheme'
        if _scheme == '':
            _scheme = 'file'
        service_class = boot._apath.cosm['services'][str(_scheme)].value
        try:
            cels += [ Dcel(address=str(urlstr), 
                        service_class=service_class) ]
        except:
            pass
root = Dcel(
    service_class=MulticelFS,
    address=cels
)

In [18]:
## Init Procedure

from Fudge import Fudge
from APath import APath
from Dcel import Dcel
from MulticelFS import MulticelFS
from urllib.parse import urlparse

bootpath = 'fs'
boot = Fudge(bootpath)
fstab = boot/'.cosm/etc/fstab'
services = boot._apath.cosm['services']
cels = []
for ea in fstab:
    if fstab/ea/'vfstype' == 'cburnfs':
        urlstr = str(fstab/ea/'spec')
        url = urlparse(urlstr)
        print('url: '+urlstr)
        print('scheme: '+url.scheme)
        print('netloc: '+url.netloc)
        print('path: '+url.path)
        _scheme = url.scheme
        if _scheme == '':
            _scheme = 'file'
        service_class = services[_scheme].value
        print(service_class)
        try:
            d = Dcel(address=str(url.path), 
                     service_class=service_class)
            print(d.listdir('/'))
            cels += [d]
        except:
            raise
        print('')
        
d = MulticelFS(cels)

print(d.listdir('.'))

CreateFailed: unable to create filesystem, 

In [ ]:
from APath import APath
from Dcel import Dcel

services = APath._rootcosm['services']

a = Dcel(address='fs', 
         service_class=services['file']
        )
print(services['file'])

In [4]:
#with CBMetaFS('fs/.cosm/var/cbmeta.zip') as m:
   #m.setinfo('boats',{"cburn":{"hosts":["raygan.com"]}})

#with CBMetaFS('fs/.cosm/var/cbmeta.zip') as m:
    #print(m.getinfo('boats').raw)
    #pass
    
from socket import gethostname



def cbtest(path='/boats/skiff.txt',
           namespaces=None
          ):
    
    print('request: '+path)
    
    hostname = gethostname()

    BlackstrapSvc.initClass()
    BlackstrapSvc.addShare(
        srcaddr = 'fs',
        hostname = hostname,
        sharename = 'fs'
    ).addShare(
        srcaddr = 'fs2',
        hostname = hostname,
        sharename = 'fs2'
    )

    
    cbDemoCosm = {
        'fs': {
            'fs': Dcel('/',BlackstrapSvc(hostname,'fs'),hostname+'/fs'),
            'fs2': Dcel('/',BlackstrapSvc(hostname,'fs2'),hostname+'/fs2'),
            #'cbmeta': Dcel('/',CBMetaFS('fs/.cosm/var/cbmeta.zip')),
        },
        "hosts": [
            "file://iph7rh/fs",
            "file://iph7rh/fs2",
            "meta:.cosm/var/cbmeta.zip"
        ],
        "services": {
            "default": OSFS,
            "file": BlackstrapSvc,
            "meta": CBMetaFS,
            "multi": MultiCelFS
        }
    }
    
    root = CBurnFS(cbDemoCosm)
    #target = root.lookup('abc/def')
    #t2 = target.lookup('ghi')
    #i = t2.getinfo('jkl')

    i = root.getinfo(path,namespaces)
    print(i.raw)
    #print(root.getinfo('boats').raw)
    #print(root.getinfo('colors').raw)
    
    if i.is_dir:
        print(str(root.listdir(path))) 
    else:
        s = root.readbytes(path)
        print(s)

    root.close()
    BlackstrapSvc.closeClass()
    

cbtest("boats")
cbtest("boats/skiff.txt")